In [1]:
import tensorflow as tf
import numpy as np

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [3]:
 train_dataset = tf.keras.utils.audio_dataset_from_directory('./data/data',
                                                             label_mode='int', 
                                                             output_sequence_length=6000,
                                                             batch_size=64
                                                             ) 

Found 10380 files belonging to 90 classes.


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(32, 6000, 1)))
model.add(Conv2D(16, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(, activation='relu'))
model.add(Dense(90, activation='softmax'))

In [5]:
model.compile('Adam', loss='BinaryCrossentropy', metrics=[tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

In [6]:
BATCH_SIZE = 32
train_dataset = train_dataset.batch(BATCH_SIZE)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
checkpoint = tf.keras.callbacks.ModelCheckpoint('/tmp/checkpoint', monitor='val_accuracy', save_best_only=True)
history = model.fit(train_dataset, epochs=20, callbacks=[callback, checkpoint], verbose=1)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 5998, 16)      160       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 5996, 16)      2320      
                                                                 
 flatten (Flatten)           (None, 2686208)           0         
                                                                 
 dense (Dense)               (None, 100)               268620900 
                                                                 
 dense_1 (Dense)             (None, 90)                9090      
                                                                 
Total params: 268,632,470
Trainable params: 268,632,470
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node 'sequential/flatten/Reshape' defined at (most recent call last):
    File "C:\Users\Blue\anaconda3\envs\tf-gpu\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Blue\anaconda3\envs\tf-gpu\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Blue\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Blue\anaconda3\envs\tf-gpu\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\Blue\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\Blue\anaconda3\envs\tf-gpu\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\Blue\anaconda3\envs\tf-gp